In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
from qiskit.primitives import sampler

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler  #Import Sampler
from qiskit_aer import Aer
from qiskit_aer import backends


In [11]:
#yf = yahoo finance
import yfinance as yf

#(NSE India stocks with .NS suffix)
stocks = ["TCS.NS", "INFY.NS", "HDFCBANK.NS", "RELIANCE.NS", "ITC.NS", "TATAMOTORS.NS", "BAJFINANCE.NS"]

# Download historical stock data for 3 years
df = yf.download(stocks, start="2021-01-01", end="2024-01-01")

#WE ARE USING ONLY THE CLOSE PRICE OF THE STOCKS
df = df['Close']
print(df.head())

[*********************100%***********************]  7 of 7 completed

Ticker      BAJFINANCE.NS  HDFCBANK.NS      INFY.NS      ITC.NS  RELIANCE.NS  \
Date                                                                           
2021-01-01    5204.156738  1368.017578  1148.968384  179.427795   898.454651   
2021-01-04    5141.127930  1359.329712  1174.309448  179.050247   899.968994   
2021-01-05    5045.326172  1369.601440  1179.368652  177.414108   888.780701   
2021-01-06    4957.902832  1363.697632  1168.703491  172.337936   865.341736   
2021-01-07    5007.873047  1359.569702  1150.517944  170.156464   863.940430   

Ticker      TATAMOTORS.NS       TCS.NS  
Date                                    
2021-01-01     185.348557  2713.367920  
2021-01-04     190.118942  2816.408447  
2021-01-05     192.007202  2866.028564  
2021-01-06     194.193619  2827.573486  
2021-01-07     195.535294  2810.246094  


In [12]:
"""we are calculating the daily return and the covariance matrix , the matrix is imp cause it tells us if our portfolio is well balanced or not
if the covariance is high then the stocks are not well balanced and if the covariance is low then the stocks are well balanced"""

returns = df.pct_change()
mean_returns = returns.mean()
cov_mat = returns.cov()


In [13]:
#building the qubo matrix
assets = len(stocks)
Q = np.zeros([assets, assets]) #init the qubo matrix
penalty  = np.mean(abs(mean_returns)) / np.mean(abs(cov_mat.values))
for i in range(assets):
    Q[i,i] = -mean_returns[i]+penalty*cov_mat.iloc[i,i]
    for j in range(i+1, assets):
        Q[i,j] = -cov_mat.iloc[i,j]
        Q[j,i] = -cov_mat.iloc[i,j]

print(Q)   


[[ 2.12947633e-03 -1.19680371e-04 -8.72513241e-05 -7.81369972e-05
  -1.11089894e-04 -1.68183378e-04 -7.11393964e-05]
 [-1.19680371e-04  1.14790747e-03 -6.88003309e-05 -4.48003523e-05
  -7.81412148e-05 -1.13221246e-04 -5.53349197e-05]
 [-8.72513241e-05 -6.88003309e-05  1.28335561e-03 -4.03060620e-05
  -7.01201913e-05 -1.03995047e-04 -1.38913389e-04]
 [-7.81369972e-05 -4.48003523e-05 -4.03060620e-05  1.01683981e-04
  -6.32902163e-05 -1.06738990e-04 -2.94255788e-05]
 [-1.11089894e-04 -7.81412148e-05 -7.01201913e-05 -6.32902163e-05
   1.09925003e-03 -1.34463145e-04 -5.77972428e-05]
 [-1.68183378e-04 -1.13221246e-04 -1.03995047e-04 -1.06738990e-04
  -1.34463145e-04  2.29030313e-03 -9.63110672e-05]
 [-7.11393964e-05 -5.53349197e-05 -1.38913389e-04 -2.94255788e-05
  -5.77972428e-05 -9.63110672e-05  8.11314365e-04]]


C:\Users\kumar\AppData\Local\Temp\ipykernel_11664\1286306283.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Q[i,i] = -mean_returns[i]+penalty*cov_mat.iloc[i,i]


In [ ]:
#building QAOA to solve qubo matrix
qubo = QuadraticProgram()
for i in range(assets):
    qubo.binary_var(f"x{i}")

#need to convert qubo to optimisation problem
linear_terms =  {f"x{i}": Q[i, i] for i in range(assets)}
quadratic_terms = {(f"x{i}", f"x{j}"): Q[i, j] for i in range(assets) for j in range(i + 1, assets)}
qubo.minimize(linear=linear_terms, quadratic=quadratic_terms)

#lets use QAOA to solve
backend = Aer.get_backend('qasm_simulator')
qaoa = QAOA(sampler=backend,optimizer=COBYLA(), reps=1)
result = qaoa.

In [28]:
help(qaoa)

Help on QAOA in module qiskit_algorithms.minimum_eigensolvers.qaoa object:

class QAOA(qiskit_algorithms.minimum_eigensolvers.sampling_vqe.SamplingVQE)
 |  QAOA(sampler: 'BaseSampler', optimizer: 'Optimizer | Minimizer', *, reps: 'int' = 1, initial_state: 'QuantumCircuit | None' = None, mixer: 'QuantumCircuit | BaseOperator' = None, initial_point: 'np.ndarray | None' = None, aggregation: 'float | Callable[[list[float]], float] | None' = None, callback: 'Callable[[int, np.ndarray, float, dict[str, Any]], None] | None' = None) -> 'None'
 |
 |  The Quantum Approximate Optimization Algorithm (QAOA).
 |
 |  QAOA is a well-known algorithm for finding approximate solutions to combinatorial-optimization
 |  problems [1].
 |
 |  The QAOA implementation directly extends :class:`.SamplingVQE` and inherits its optimization
 |  structure. However, unlike VQE, which can be configured with arbitrary ansatzes, QAOA uses its
 |  own fine-tuned ansatz, which comprises :math:`p` parameterized global :mat